# Setup and Imports

Import useful libraries for defining our model, set up the model

In [ ]:
from rdflib import Namespace, Graph
from buildingmotif import BuildingMOTIF
from buildingmotif.namespaces import BRICK
from buildingmotif.dataclasses import Model, Library

In [ ]:
bm = BuildingMOTIF("sqlite://", "topquadrant") # in-memory

In [ ]:
BLDG = Namespace("urn:my_site/")
model = Model.create(BLDG)

In [ ]:
qudtqk = Library.load(ontology_graph="http://qudt.org/2.1/vocab/quantitykind")
qudtunit = Library.load(ontology_graph="http://qudt.org/2.1/vocab/unit")
constraints = Library.load(ontology_graph="constraints/constraints.ttl")
brick = Library.load(ontology_graph="../libraries/brick/Brick.ttl")
g36 = Library.load(directory="../libraries/ashrae/guideline36")

- simulate looking at a mechanical schedule:
    - see number and type of equipment
    - **our example**: 1 AHU, 2 RVAVs
- now look at libraries to see if there are some useful "shapes"
    - based on forecasted app requirements
    - use these to inform model creation

In [ ]:
sc = g36.get_shape_collection()

In [ ]:
sc.get_shapes_about_class(BRICK.AHU)

In [ ]:
sc.get_shapes_about_class(BRICK.RVAV)

# Write site manifest

- write a "manifest" to encode our requirements:
    - 1 AHU
    - 2 RVAVs; should run G36 sequence of operations
- this is the *low level* view -- will be abstracted by us and other developers

In [ ]:
%%bash
cat <<EOF > my_site.ttl
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix constraint: <https://nrel.gov/BuildingMOTIF/constraints#> .
@prefix : <urn:my_site_constraints/> .

: a owl:Ontology ;
    owl:imports <https://brickschema.org/schema/1.4/Brick> ;
    owl:imports <https://nrel.gov/BuildingMOTIF/constraints> ;
    owl:imports <urn:ashrae/g36> .

# read off of mechanical schedule
:vav-count a sh:NodeShape ;
    sh:message "Need 2 VAVs" ;
    sh:targetNode : ; # target *this* graph
    constraint:exactCount 2 ;
    constraint:class <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .

:ahu-count a sh:NodeShape ;
    sh:message "Need 1 AHU" ;
    sh:targetNode : ; # target *this* graph
    constraint:exactCount 1 ;
    constraint:class brick:AHU .
EOF

Now we load our manifest into BuildingMOTIF

In [ ]:
manifest = Library.load(ontology_graph="my_site.ttl")

# Iterative model creation

- validate model to determine if it is *semantically sufficient*
- if this fails, BuildingMOTIF will use resulting report to help you patch the model

In [ ]:
# pass our collections of shapes in to validate the model
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), 
                      g36.get_shape_collection(), brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Model is valid? {ctx.valid}")
# higher-level descriptions of each of the violations
print("Reasons why:")
for focus_node, diffs in ctx.diffset.items():
    print(focus_node)
    for diff in diffs:
        print("  - " + diff.reason())

- some constraints can be interpreted by BuildingMOTIF
- BuildingMOTIF identifies common patterns
    - presents these as new templates to the user
    - templates provide structure to input from user/other software

In [ ]:
generated_templates = ctx.as_templates()
print(generated_templates)
for t in generated_templates:
    print('-'*80)
    print(t.body.serialize())
    ident = input("Give value for 'name' in the above template: ")
    model.add_graph(t.evaluate({"name": BLDG[ident]}))

In [ ]:
# now we can look at our model ( so far )
print(model.graph.serialize())

In [ ]:
# validate it again!
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), g36.get_shape_collection(), brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Model is valid? {ctx.valid}")
print("Reasons why:")
for focus_node, diffs in ctx.diffset.items():
    print(focus_node)
    for diff in diffs:
        print("  - " + diff.reason())

- model is still failing, so we ask BuildingMOTIF for more feedback
- ask user for more inputs on some of the missing sensors

- BuildingMOTIF can add missing metadata automatically
    - if unambiguous (true for some constraints)
    - otherwise, attempt "autocomplete" from existing model

In [ ]:
print(model.graph.serialize())

In [ ]:
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), g36.get_shape_collection(), brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Is model valid yet? {ctx.valid}")

In [ ]:
print(ctx.report.serialize())

 - we can also invoke templates ourselves, rather than having BuildingMOTIF suggest them
 - here using `fill()` as a debug trick to avoid having to give inputs

In [ ]:
# use templates to finish it up!
vav_templ = g36.get_template_by_name("vav-with-reheat").inline_dependencies()
_, graph = vav_templ.evaluate({"name": BLDG["vav1"]}).fill(BLDG)
model.add_graph(graph)
_, graph = vav_templ.evaluate({"name": BLDG["vav2"]}).fill(BLDG)
model.add_graph(graph)

In [ ]:
ctx = model.validate([manifest.get_shape_collection(),
                      constraints.get_shape_collection(),
                      g36.get_shape_collection(),
                      brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Is model valid yet? {ctx.valid}")

In [ ]:
print(model.graph.serialize())

In [ ]:
print(ctx.report.serialize())